In [103]:
#r "nuget: XPlot.Plotly, 4.0.6"

open XPlot.Plotly
open System

Installed Packages XPlot.Plotly, 4.0.6

In [104]:

type CandleData = {
    Date: DateTime
    Open: float
    Close: float
    High: float
    Low: float
}

type TilsonData = {
    Candle: CandleData
    TilsonValue: float
}

type BollingerData = {
    Candle: CandleData
    UpperBand: float
    LowerBand: float
    MiddleBand: float
}


type CandleVariant =
    | CandleV of CandleData
    | TilsonV of TilsonData
    | BollingerV of BollingerData


In [105]:
open System
open System.Globalization

let parseBollinger filePath: BollingerData[] =
    let lines = System.IO.File.ReadAllLines(filePath)
    lines
    |> Array.skip 1
    |> Array.map (fun line ->
        let columns = line.Split(',')
        let epoch = Int64.Parse(columns.[0])
        let date = DateTimeOffset.FromUnixTimeMilliseconds(epoch).DateTime
        {
            Candle = { Date = date; Open = float columns.[1]; High = float columns.[2]; Low = float columns.[3]; Close = float columns.[4] }
            UpperBand = float columns.[5]
            LowerBand = float columns.[6]
            MiddleBand = float columns.[7]
        })


let parseTilson filePath: TilsonData[] =
    let lines = System.IO.File.ReadAllLines(filePath)
    lines
    |> Array.skip 1
    |> Array.map (fun line ->
        let columns = line.Split(',')
        let epoch = Int64.Parse(columns.[0])
        let date = DateTimeOffset.FromUnixTimeMilliseconds(epoch).DateTime
        {
            Candle = { Date = date; Open = float columns.[1]; High = float columns.[2]; Low = float columns.[3]; Close = float columns.[4] }
            TilsonValue = float columns.[5]
        })


In [106]:
let extractCandleData (variant: CandleVariant) = 
    match variant with
        | CandleV c -> c
        | TilsonV t -> t.Candle
        | BollingerV b -> b.Candle

let getCandleStickTrace (records: CandleVariant[]) : Trace =
    Candlestick(
        x = (records |> Array.map (fun r -> (extractCandleData r).Date.ToString("yyyy-MM-dd HH:mm:ss"))),
        ``open`` = (records |> Array.map (fun r -> (extractCandleData r).Open)),
        high = (records |> Array.map (fun r -> (extractCandleData r).High)),
        low = (records |> Array.map (fun r -> (extractCandleData r).Low)),
        close = (records |> Array.map (fun r -> (extractCandleData r).Close)),
        name = "Candlestick"
    )

In [107]:
let getBollingerUpperLine (records: BollingerData[]) = 
    Scatter(
        x = (records |> Array.map (fun r -> r.Candle.Date.ToString("yyyy-MM-dd HH:mm:ss"))),
        y = (records |> Array.map (fun r -> r.UpperBand)),
        name = "bollinger_upper_line",
        yaxis = "y2"
    )

let getBollingerLowerLine (records: BollingerData[]) = 
    Scatter(
        x = (records |> Array.map (fun r -> r.Candle.Date.ToString("yyyy-MM-dd HH:mm:ss"))),
        y = (records |> Array.map (fun r -> r.LowerBand)),
        name = "bollinger_lower_line",
        yaxis = "y2"
    )


let getBollingerMiddleLine (records: BollingerData[]) = 
    Scatter(
        x = (records |> Array.map (fun r -> r.Candle.Date.ToString("yyyy-MM-dd HH:mm:ss"))),
        y = (records |> Array.map (fun r -> r.MiddleBand)),
        name = "bollinger_middle_line",
        yaxis = "y2"
    )

In [108]:
let getTilsonLine (records: TilsonData[]) =
    Scatter(
        x = (records |> Array.map (fun r -> r.Candle.Date.ToString("yyyy-MM-dd HH:mm:ss"))),
        y = (records |> Array.map (fun r -> r.TilsonValue)),
        name = "tilson_line",
        yaxis = "y2"
    )

In [109]:

// Define file paths
let bollingerFile = Environment.CurrentDirectory + "/bollinger.csv"
let tilsonFile = Environment.CurrentDirectory + "/tilson.csv"

// Read and parse the records
let bollingerRecords = parseBollinger bollingerFile
let tilsonRecords = parseTilson tilsonFile

In [110]:
let traces = ResizeArray<Trace>()

let candleStickData = getCandleStickTrace (bollingerRecords |> Array.map (fun r -> BollingerV r))
traces.Add(candleStickData)

traces.Add(getTilsonLine tilsonRecords)

let bollingerUpperLine = getBollingerUpperLine bollingerRecords
let bollingerLowerLine = getBollingerLowerLine bollingerRecords
let bollingerMiddleLine = getBollingerMiddleLine bollingerRecords

traces.Add(bollingerUpperLine)
traces.Add(bollingerLowerLine)
traces.Add(bollingerMiddleLine)

let chart = Chart.Plot(traces)
chart.WithLayout(Layout.Layout(
    title = "Indicators",
    yaxis = Yaxis(title = "Price"),
    yaxis2 = Yaxis(title = "ChannelTilson", overlaying = "y", side = "right")
))

chart.Show()